In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [ ]:
## import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as plt

# librosa is a Python library for analyzing audio and music.
# It can be used to extract the data from the audio files we will see it later
import librosa
import torch
import librosa.display
import warnings
warnings.filterwarnings("ignore")
# to play the audio files
from IPython.display import Audio
plt.style.use('seaborn-white')

In [ ]:
! pip install transformers

In [ ]:
pip install speechbrain

In [ ]:
crema = "/kaggle/input/cremad/AudioWAV/"
crema_directory_list = os.listdir(crema)
file_name = []
file_emotion = []
file_path = []
file_id = []

for file in crema_directory_list:
    # storing file paths
    file_name.append(file.split('.')[0])
    file_path.append(crema + file)
    # storing file emotions
    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('Sadness')
    elif part[2] == 'ANG':
        file_emotion.append('Anger')
    elif part[2] == 'FEA':
        file_emotion.append('Fear')
    elif part[2] == 'HAP':
        file_emotion.append('Happiness')
    elif part[2] == 'NEU':
        file_emotion.append('Neutral')
    elif part[2] == 'DIS':
        file_emotion.append('Disgust')
    else:
        file_emotion.append('Unknown')
    
    
    file_id.append(int(part[0]))
   

filename_df = pd.DataFrame(file_name, columns=['Name'])
path_df = pd.DataFrame(file_path, columns=['Path'])
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
speaker_id_df = pd.DataFrame(file_id, columns=['ActorID'])

Crema_df = pd.concat([filename_df,path_df,emotion_df, speaker_id_df], axis=1)
Crema_df['source'] = 'cremad'
cdataset = Crema_df
cemotions = ['Sadness', 'Happiness', 'Anger', 'Fear','Disgust', 'Neutral']
cdataset = cdataset[cdataset['Emotions'].isin(cemotions)].reset_index(drop = True)
cdataset['Emotions'].value_counts()

In [ ]:

demographis_df = pd.read_csv('/kaggle/input/cremad-demographics/VideoDemographics.csv')

cdata=cdataset.merge(demographis_df, left_on='ActorID', right_on='ActorID')

In [ ]:
cdata.drop(labels={'Race','Ethnicity'}, axis=1, inplace=True)

In [ ]:
cdata

In [ ]:
 cdata_transformed=cdata.copy()

In [ ]:
cdata_transformed

In [ ]:
from sklearn import preprocessing
lec = preprocessing.LabelEncoder()
cdata_transformed['Emotions'] = lec.fit_transform(cdata['Emotions'])
lec_name_mapping = dict(zip(lec.classes_, lec.transform(lec.classes_)))
print(lec_name_mapping)

In [ ]:
hubert=np.load('/kaggle/input/final-cremad-embeddings/CREMA-D-EMBED/CREMAD_hubert_embeddings.npy')
mfcc=np.load('/kaggle/input/final-cremad-embeddings/CREMA-D-EMBED/CREMAD_mfcc_embeddings.npy')
mms=np.load('/kaggle/input/final-cremad-embeddings/CREMA-D-EMBED/CREMAD_mms_embeddings.npy')
trillson=np.load('/kaggle/input/final-cremad-embeddings/CREMA-D-EMBED/CREMAD_trillsson_embeddings.npy')
unispeech=np.load('/kaggle/input/final-cremad-embeddings/CREMA-D-EMBED/CREMAD_unispeech_embeddings.npy')
w2v2=np.load('/kaggle/input/final-cremad-embeddings/CREMA-D-EMBED/CREMAD_w2v2_embeddings.npy')
wavlm=np.load('/kaggle/input/final-cremad-embeddings/CREMA-D-EMBED/CREMAD_wavlm_embeddings.npy')
xlsr=np.load('/kaggle/input/final-cremad-embeddings/CREMA-D-EMBED/CREMAD_xlsr_embeddings.npy')
xv=np.load('/kaggle/input/final-cremad-embeddings/CREMA-D-EMBED/CREMAD_xv_embeddings.npy')

In [ ]:
from sklearn import preprocessing
lec = preprocessing.LabelEncoder()
cdata_transformed['Age'] = lec.fit_transform(cdata['Age'])
lec_name_mapping = dict(zip(lec.classes_, lec.transform(lec.classes_)))
print(lec_name_mapping)

In [ ]:
from sklearn import preprocessing
lec = preprocessing.LabelEncoder()
cdata_transformed['ActorID'] = lec.fit_transform(cdata['ActorID'])
lec_name_mapping = dict(zip(lec.classes_, lec.transform(lec.classes_)))
print(lec_name_mapping)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cdata_transformed['Sex'] = lec.fit_transform(cdata['Sex'])


In [ ]:
cdata

In [ ]:
cdata_outputs =cdata_transformed[['ActorID', 'Age', 'Sex', 'Emotions']]

In [ ]:
cdata_outputs

In [ ]:

# Train, val and test split
from sklearn.model_selection import train_test_split, cross_val_predict

# First, split data into training and testing sets (80% training, 20% testing)
x_train, x_val, y_train, y_val = train_test_split(w2v2, cdata_outputs['Sex'], test_size=0.2, stratify=cdata_outputs['Sex'], random_state=0)


print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

In [ ]:
y_train

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
y_val.nunique()

In [ ]:
trillson.shape

In [ ]:
trillson_emb = trillson.reshape(trillson.shape[0], -1)

In [ ]:
mkdir trillson

In [ ]:
pwd

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE

# Assuming 'trillson_emb' is your embedding data
tsne = TSNE(random_state=12, n_iter=2000, metric="cosine")
tsne_proj = tsne.fit_transform(trillson_emb)

# Plotting using TSNE
fig, ax = plt.subplots(figsize=(8, 8))
labels = ['Male', 'Female']

# Assign colors manually or using a colormap
colors = ['blue', 'red']

for i, lab in enumerate(labels):
    indices = cdata['Sex'] == lab
    ax.scatter(tsne_proj[indices, 0], tsne_proj[indices, 1], c=colors[i], label=lab, alpha=0.5)

ax.legend(fontsize='large', markerscale=2)
plt.savefig("./trillson/gender")
plt.show()


In [ ]:
# Reference t-SNE: https://www.kaggle.com/code/colinmorris/visualizing-embeddings-with-t-sne
from sklearn.manifold import TSNE

# The default of 1,000 iterations gives fine results, but I'm training for longer just to eke
# out some marginal improvements. NB: This takes almost an hour!
tsne = TSNE(random_state=12, n_iter=2000, metric="cosine")

tsne_proj = tsne.fit_transform(trillson_emb)
# Plot those points as a scatter plot and label them based on the pred labels
from matplotlib import cm
cmap = cm.get_cmap('tab20')
fig, ax = plt.subplots(figsize=(8,8))
num_categories =6

labels =  ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness']
for i,lab in enumerate(labels):
    indices = cdata['Emotions']==lab
    ax.scatter(tsne_proj[indices,0],tsne_proj[indices,1], c=np.array(cmap(i)).reshape(1,4), label = lab ,alpha=0.5)
ax.legend(fontsize='large', markerscale=2)
plt.savefig("./trillson/emotions")
plt.show()

In [ ]:
from keras.layers import Input, Conv1D, Dense, concatenate, Flatten
from keras.models import Model
import tensorflow as tf

def create_model(input_speakrec):

    x1 = Conv1D(32, 3, activation='relu', padding='same')(input_speakrec)
    x1 = tf.keras.layers.MaxPooling1D()(x1)
    x1 = Conv1D(64, 3, activation='relu', padding='same')(x1)
    x1 = tf.keras.layers.MaxPooling1D()(x1)
    x1 = Flatten()(x1)


    x = Dense(200, activation='relu')(x1)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = Dense(56, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.005))(x)
    return x
    

def final_model(inp1):

    x = create_model(inp1)
    sid = Dense(2, activation='softmax', name='Gender')(x)
    model = Model(inputs=inp1, outputs=sid) 

    return model

input_speakrec = Input(shape=[768, 1])


model = final_model(input_speakrec)
model.summary()


In [ ]:
# from keras.layers import Input, Conv1D, Dense, concatenate, Flatten
# from keras.models import Model
# import tensorflow as tf

# def create_model(input_speakrec):
 
    
    
#     # Define the CNN layers for each modality
#     x1 = Conv1D(32, 3, activation='relu', padding='same')(input_speakrec)
#     x1 = tf.keras.layers.MaxPooling1D()(x1)
#     x1 = Conv1D(64, 3, activation='relu', padding='same')(x1)
#     x1 = tf.keras.layers.MaxPooling1D()(x1)
#     x1 = Flatten()(x1)

# #     x2 = Conv1D(32, 3, activation='relu', padding='same')(input_mfcc)
# #     x2 = tf.keras.layers.MaxPooling1D()(x2)
# #     x2 = Conv1D(64, 3, activation='relu', padding='same')(x2)
# #     x2 = tf.keras.layers.MaxPooling1D()(x2)
# #     x2 = Flatten()(x2)

# #     x3 = Conv1D(32, 3, activation='relu', padding='same')(input_conformer)
# #     x3 = tf.keras.layers.MaxPooling1D()(x3)
# #     x3 = Conv1D(64, 3, activation='relu', padding='same')(x3)
# #     x3 = tf.keras.layers.MaxPooling1D()(x3)
# #     x3 = Flatten()(x3)

# #     # Concatenate the features from different modalities
# #     concat = concatenate([x1, x2], axis=1)


#     x = Dense(200, activation='relu')(x1)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = Dense(64, activation='relu')(x)
#     x = tf.keras.layers.Dropout(0.1)(x)
#     x = Dense(56, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
#     return x
    

# def final_model(inp1):

#     x = create_model(inp1)
#     sid = Dense(91, activation='softmax', name='ActorID')(x)
#     emo = Dense(6, activation='softmax', name='Emotions')(x)
#     age = Dense(38, activation='softmax', name='Age')(x)
#     gender = Dense(2, activation='softmax', name='Gender')(x)
#     model = Model(inputs=inp1, outputs=[sid, emo, age, gender])  # Specify multiple inputs as a list

#     return model

# input_speakrec = Input(shape=[1024, 1])


# # Create the model
# model = final_model(input_speakrec)
# model.summary()


In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
lr = 1e-3
import tensorflow_addons as tfa
# optimizer = tfa.optimizers.RectifiedAdam(learning_rate= lr)
optimizer = tf.keras.optimizers.Adam(learning_rate= lr)
# Compile the model with the Riemannian optimizer            
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  #from_logits=True
    metrics= ['accuracy']  #[tf.keras.metrics.SparseCategoricalAccuracy()],
)


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                 factor = 0.2,
                                                 patience = 1,
                                                 verbose = 1,
                                                 min_delta = 1e-4,
                                                 min_lr = 1e-15,
                                                 mode = 'max')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 70,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = './cnn_cremad.hdf5',
                                                  monitor = 'val_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')

callbacks = [earlystopping, checkpointer, reduce_lr]#reduce_lr]

In [ ]:
# history = model.fit(
#     x_train,
#     {'ActorID': y_train['ActorID'], 'Emotions': y_train['Emotions'], 'Age': y_train['Age'],'Gender': y_train['Sex']},
#     validation_data=(x_val, {'ActorID': y_val['ActorID'], 'Emotions': y_val['Emotions'], 'Age': y_val['Age'],'Gender': y_val['Sex']}),
#     batch_size=64,
#     epochs=150,
#     callbacks=callbacks
# )

In [ ]:
pip install h5py

In [ ]:
import time

start_time = time.time()
history = model.fit( x_train, y_train,
                   validation_data =  (x_val, y_val),
                   batch_size = 32,
                   epochs = 15,
                   callbacks = callbacks)


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# plt.plot(history.history['val_ActorID_accuracy'])
import matplotlib.pyplot as plt
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
y_val

In [ ]:
from sklearn.metrics import classification_report


# Make predictions on the validation data
predictions = model.predict(x_val)
y_predmax = tf.math.argmax(predictions, axis = 1)



# Calculate classification reports for each output
classification_report_speaker_id = classification_report(
   y_val,
    y_predmax, digits=4  # Corrected indexing for speaker_id
#     target_names=target_names_speaker_id
)

# classification_report_emotions = classification_report(
#     target_names_emotion,
#     y_predmax2  # Corrected indexing for Emotion-labels
# #     target_names=target_names_emotion_labels
# )

# classification_report_age = classification_report(
#     target_names_age,
#     y_predmax3  # Corrected indexing for Emotion-labels
# #     target_names=target_names_emotion_labels
# )

# classification_report_sex = classification_report(
#     target_names_gender,
#     y_predmax4  # Corrected indexing for Emotion-labels
# #     target_names=target_names_emotion_labels
# )

# Print the classification reports
print("Classification Report for Speaker ID:")
print(classification_report_speaker_id)

# print("\nClassification Report for Emotion Labels:")
# print(classification_report_emotions)


# print("\nClassification Report for age:")
# print(classification_report_age)

# print("\nClassification Report for Gender:")
# print(classification_report_sex)

> # #  Age

In [ ]:

# Train, val and test split
from sklearn.model_selection import train_test_split, cross_val_predict

# First, split data into training and testing sets (80% training, 20% testing)
x_train, x_val, y_train, y_val = train_test_split(unispeech, cdata_outputs['Age'], test_size=0.2, stratify=cdata_outputs['Age'], random_state=99)


print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

In [ ]:
from keras.layers import Input, Conv1D, Dense, concatenate, Flatten
from keras.models import Model
import tensorflow as tf

def create_model(input_speakrec):

    x1 = Conv1D(32, 3, activation='relu', padding='same')(input_speakrec)
    x1 = tf.keras.layers.MaxPooling1D()(x1)
    x1 = Conv1D(64, 3, activation='relu', padding='same')(x1)
    x1 = tf.keras.layers.MaxPooling1D()(x1)
    x1 = Flatten()(x1)


    x = Dense(156, activation='relu')(x1)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = Dense(56, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.005))(x)
    return x
    

def final_model(inp1):

    x = create_model(inp1)
    age = Dense(1, activation='linear', name='Age')(x)
    model = Model(inputs=inp1, outputs=age) 

    return model

input_speakrec = Input(shape=[768, 1])


model = final_model(input_speakrec)
model.summary()


In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
lr = 1e-3
import tensorflow_addons as tfa
# optimizer = tfa.optimizers.RectifiedAdam(learning_rate= lr)
optimizer = tf.keras.optimizers.Adam(learning_rate= lr)
# Compile the model with the Riemannian optimizer            
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.MeanSquaredError(),  #from_logits=True
    metrics= tf.keras.metrics.RootMeanSquaredError()  #[tf.keras.metrics.SparseCategoricalAccuracy()],
)


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                 factor = 0.2,
                                                 patience = 1,
                                                 verbose = 1,
                                                 min_delta = 1e-4,
                                                 min_lr = 1e-15,
                                                 mode = 'max')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 70,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = './cnn_cremad.hdf5',
                                                  monitor = 'val_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')

callbacks = [earlystopping, checkpointer, reduce_lr]#reduce_lr]

In [ ]:
import time

start_time = time.time()
history = model.fit( x_train, y_train,
                   validation_data =  (x_val, y_val),
                   batch_size = 32,
                   epochs = 50,
                   callbacks = callbacks)


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
from sklearn.metrics import  mean_squared_error
predictions_age=model.predict(x_val)
mse_age = mean_squared_error(y_val, predictions_age,squared=False)
print(mse_age)
from sklearn.metrics import  mean_absolute_error
mae_age = mean_absolute_error(y_val, predictions_age)
print(mae_age)

In [ ]:
history = model.fit(
    x_train,
    {'ActorID': y_train['ActorID'], 'Emotions': y_train['Emotions'], 'Age': y_train['Age'],'Sex': y_train['Sex']},
    validation_data=(x_val,  {'ActorID': y_val['ActorID'], 'Emotions': y_val['Emotions'], 'Age': y_val['Age'],'Sex': y_val['Sex']}),
    batch_size=32,
    epochs=50,
    callbacks=callbacks
)

In [ ]:
from sklearn.metrics import classification_report

# Assuming you've already trained your model as mentioned in your code
# ...

# Make predictions on the validation data
predictions = model.predict(x_val)
y_predmax = tf.math.argmax(predictions[0], axis = 1)
y_predmax2 = tf.math.argmax(predictions[1], axis = 1)
y_predmax3 = tf.math.argmax(predictions[2], axis = 1)
y_predmax4 = tf.math.argmax(predictions[3], axis = 1)
# Define the target names for each output
target_names_speaker_id = y_val['ActorID']  # Replace with your actual class labels for speaker_id
target_names_emotion = y_val['Emotions'] 
target_names_age = y_val['Age'] 
target_names_gender = y_val['Sex'] # Replace with your actual class labels for Emotion-labels

# Calculate classification reports for each output
classification_report_speaker_id = classification_report(
   target_names_speaker_id,
    y_predmax  # Corrected indexing for speaker_id
#     target_names=target_names_speaker_id
)

classification_report_emotions = classification_report(
    target_names_emotion,
    y_predmax2  # Corrected indexing for Emotion-labels
#     target_names=target_names_emotion_labels
)

classification_report_age = classification_report(
    target_names_age,
    y_predmax3  # Corrected indexing for Emotion-labels
#     target_names=target_names_emotion_labels
)

classification_report_sex = classification_report(
    target_names_gender,
    y_predmax4  # Corrected indexing for Emotion-labels
#     target_names=target_names_emotion_labels
)

# Print the classification reports
print("Classification Report for Speaker ID:")
print(classification_report_speaker_id)

print("\nClassification Report for Emotion Labels:")
print(classification_report_emotions)


print("\nClassification Report for age:")
print(classification_report_age)

print("\nClassification Report for Gender:")
print(classification_report_sex)

In [ ]:
# Reference t-SNE: https://www.kaggle.com/code/colinmorris/visualizing-embeddings-with-t-sne
from sklearn.manifold import TSNE

# The default of 1,000 iterations gives fine results, but I'm training for longer just to eke
# out some marginal improvements. NB: This takes almost an hour!
tsne = TSNE(random_state=1, n_iter=1000, metric="cosine")

tsne_proj = tsne.fit_transform(waveform_embeddings)
# Plot those points as a scatter plot and label them based on the pred labels
from matplotlib import cm
cmap = cm.get_cmap('tab20')
fig, ax = plt.subplots(figsize=(8,8))
num_categories = 6
# {'Anger': 0, 'Disgust': 1, 'Fear': 2, 'Happiness': 3, 'Neutral': 4, 'Sadness': 5}
labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness']
for lab in range(num_categories):
    indices = cdataset['labels']==lab
    ax.scatter(tsne_proj[indices,0],tsne_proj[indices,1], c=np.array(cmap(lab)).reshape(1,4), label = labels[lab] ,alpha=0.5)
ax.legend(fontsize='large', markerscale=2)
plt.savefig("./unispeechsat_cremad_tsne")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Assuming you've already trained your model as mentioned in your code
# ...

# Make predictions on the validation data
predictions = model.predict(x_test)
y_predmax = tf.math.argmax(predictions[0], axis = 1)
y_predmax2 = tf.math.argmax(predictions[1], axis = 1)
y_predmax3 = tf.math.argmax(predictions[2], axis = 1)
y_predmax4 = tf.math.argmax(predictions[3], axis = 1)
# Define the target names for each output
target_names_speaker_id = y_test['ActorID']  # Replace with your actual class labels for speaker_id
target_names_emotion = y_test['Emotions'] 
target_names_age = y_test['Age'] 
target_names_gender = y_test['Sex'] # Replace with your actual class labels for Emotion-labels

# Calculate classification reports for each output
classification_report_speaker_id = classification_report(
   target_names_speaker_id,
    y_predmax  # Corrected indexing for speaker_id
#     target_names=target_names_speaker_id
)

classification_report_emotions = classification_report(
    target_names_emotion,
    y_predmax2  # Corrected indexing for Emotion-labels
#     target_names=target_names_emotion_labels
)

classification_report_age = classification_report(
    target_names_age,
    y_predmax3  # Corrected indexing for Emotion-labels
#     target_names=target_names_emotion_labels
)

classification_report_sex = classification_report(
    target_names_gender,
    y_predmax4  # Corrected indexing for Emotion-labels
#     target_names=target_names_emotion_labels
)

# Print the classification reports
print("Classification Report for Speaker ID:")
print(classification_report_speaker_id)

print("\nClassification Report for Emotion Labels:")
print(classification_report_emotions)


print("\nClassification Report for age:")
print(classification_report_age)

print("\nClassification Report for Gender:")
print(classification_report_sex)

